# **Data Profiling and Quality Report**


---



####1. Import Libraries: Load Pandas and NumPy for data handling.

####2. Load Dataset: Read CSV into df (low_memory=False for large files).

####3. Preview Data: df.head() shows the first few rows.

####4. Dataset Size: df.shape gives rows and columns.

####5. Column Info: df.dtypes shows data types.

####6. Missing Values: df.isnull().sum() counts NaNs per column.

####7. Missing Data Map: df.isna() highlights where values are missing.

In [1]:
# Import Libraries
import numpy as np
import pandas as pd

In [2]:
# Load your log file
df = pd.read_csv("/content/logs_preprocessed.csv", low_memory=False)

In [3]:
print(df.head()) # head() => used to display first five rows in dataset

                                  filename    dut dut_version os_version  \
0  tc_func_tcp_tfg_001_20240722-113936.log  Cisco         1.2      Linux   
1  tc_func_tcp_tfg_001_20240722-113936.log  Cisco         1.2      Linux   
2  tc_func_tcp_tfg_001_20240722-114249.log  Cisco         1.2      Linux   
3  tc_func_tcp_tfg_001_20240722-114249.log  Cisco         1.2      Linux   
4  tc_func_tcp_tfg_001_20240722-114343.log  Cisco         1.2      Linux   

      config             test_case_id  line_number                timestamp  \
0  tcp_cisco  tc_func_tcp_tfg_001.tcl          568  2024-07-22 11:40:13.618   
1  tcp_cisco  tc_func_tcp_tfg_001.tcl          693  2024-07-22 11:40:18.642   
2  tcp_cisco  tc_func_tcp_tfg_001.tcl          566  2024-07-22 11:43:16.338   
3  tcp_cisco  tc_func_tcp_tfg_001.tcl          691  2024-07-22 11:43:21.375   
4  tcp_cisco  tc_func_tcp_tfg_001.tcl          566  2024-07-22 11:44:10.802   

     run_date status                                          error_

In [4]:
print("Dataset Shape:", df.shape) # Display the rows and columns
print("\nColumn Info:") # print the statement
print(df.dtypes) # Print the datatypes of the column

Dataset Shape: (8605, 13)

Column Info:
filename        object
dut             object
dut_version     object
os_version      object
config          object
test_case_id    object
line_number      int64
timestamp       object
run_date        object
status          object
error_msg       object
suite           object
raw_line        object
dtype: object


In [5]:
print(df.isnull().sum()) # Identify the Missing values


filename        0
dut             0
dut_version     0
os_version      0
config          0
test_case_id    0
line_number     0
timestamp       0
run_date        0
status          0
error_msg       0
suite           0
raw_line        0
dtype: int64


##**Coverage,Missing Values, and Duplicate Values**

---
####This code generates a detailed missing values report for a DataFrame by counting the number and percentage of missing entries in each column. It shows which columns have missing data and to what extent, helping identify where data cleaning or imputation is needed.


In [6]:
# Calculate the count of missing (NA) values per column
missing_report = df.isna().sum().reset_index()

# Rename columns for clear display: 'Column' for column names, 'Missing_Count' for missing value counts
missing_report.columns = ["Column","Missing_Count"]

# Calculate the percentage of missing values relative to total rows for each column
missing_report["Missing_%"] = (missing_report["Missing_Count"] / len(df)) * 100

# Print a report sorted by descending missing percentage to highlight worst columns first
print("\nMissing Values Report:")
print(missing_report.sort_values("Missing_%", ascending=False))



Missing Values Report:
          Column  Missing_Count  Missing_%
0       filename              0        0.0
1            dut              0        0.0
2    dut_version              0        0.0
3     os_version              0        0.0
4         config              0        0.0
5   test_case_id              0        0.0
6    line_number              0        0.0
7      timestamp              0        0.0
8       run_date              0        0.0
9         status              0        0.0
10     error_msg              0        0.0
11         suite              0        0.0
12      raw_line              0        0.0


In [7]:
# Count the number of duplicate rows in the DataFrame
duplicate_count = df.duplicated().sum()

# Print the total count of duplicate rows found
print(f"\nDuplicate Rows: {duplicate_count}")



Duplicate Rows: 0


####This code calculates data coverage per column in the DataFrame by counting non-missing values and computing their percentage relative to the total rows. It then prints a report sorted by coverage percentage, helping identify columns with incomplete data coverage.

In [8]:
# Create a new DataFrame summarizing columns, count of non-missing values, and coverage percentage
coverage_report = pd.DataFrame({
    "Column": df.columns,
    "NonNull_Count": df.notna().sum(),
    "Coverage_%": (df.notna().sum() / len(df)) * 100
})

# Print a heading for the coverage report
print("\nData Coverage:")

# Print the coverage report sorted by lowest coverage percentage first
print(coverage_report.sort_values("Coverage_%"))



Data Coverage:
                    Column  NonNull_Count  Coverage_%
filename          filename           8605       100.0
dut                    dut           8605       100.0
dut_version    dut_version           8605       100.0
os_version      os_version           8605       100.0
config              config           8605       100.0
test_case_id  test_case_id           8605       100.0
line_number    line_number           8605       100.0
timestamp        timestamp           8605       100.0
run_date          run_date           8605       100.0
status              status           8605       100.0
error_msg        error_msg           8605       100.0
suite                suite           8605       100.0
raw_line          raw_line           8605       100.0


## **Critical Fields**

---

####This code filters the coverage report to focus only on a set of critical columns important for machine learning readiness. It then prints the coverage details for these selected critical fields, helping quickly assess their data completeness.

In [13]:
# Define a list of critical columns needed for ML or core analysis
critical_fields = ["test_case_id", "dut", "suite", "config", "error_msg", "status", "run_date", "timestamp"]

# Filter the coverage report DataFrame to include only these critical columns
critical_report = coverage_report[coverage_report["Column"].isin(critical_fields)]

# Print the coverage details for the critical fields
print("\nCritical Fields Coverage (ML readiness):")
print(critical_report)



Critical Fields Coverage (ML readiness):
                    Column  NonNull_Count  Coverage_%
dut                    dut           8605       100.0
config              config           8605       100.0
test_case_id  test_case_id           8605       100.0
timestamp        timestamp           8605       100.0
run_date          run_date           8605       100.0
status              status           8605       100.0
error_msg        error_msg           8605       100.0
suite                suite           8605       100.0


##**Identify Gaps**

---
#### This code checks for potential data quality issues and prints warnings about key areas that need attention. It highlights missing values in important columns such as DUT version, OS version, and error messages, and warns if duplicate rows are present. The final print statement marks the completion of the profiling step.


In [14]:
# Print header for missing data and quality gaps report
print("\nPotential Gaps to Fix:")

# Check if 'dut_version' column exists and has missing values, warn about its reliability
if "dut_version" in df.columns:
    print("- DUT Version has missing values → may not be reliable for ML.")

# Check if 'os_version' exists and has missing values, print count needing better extraction
if "os_version" in df.columns and df["os_version"].isna().sum() > 0:
    print(f"- OS Version missing in {df['os_version'].isna().sum()} rows → need better extraction.")

# Warn if any logs have missing error messages
if df["error_msg"].isna().sum() > 0:
    print("- Some logs have no error messages.")

# Warn if any duplicate rows exist in dataset
if duplicate_count > 0:
    print("- Dataset has duplicate rows, consider removing them.")

# Indicate profiling process is complete
print("\nProfiling Complete.")



Potential Gaps to Fix:
- DUT Version has missing values → may not be reliable for ML.

Profiling Complete.
